In [1]:
import os
import warnings
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import SVR
from tensorflow.keras.layers import Dense, InputLayer, Dropout
from tensorflow.keras.models import model_from_json, load_model

warnings.filterwarnings('ignore')
NUM_SAMPLES = 5

In [3]:
def initial_process(_input_dir, _output_dir="output"):
    """
    :param _input_dir: input directory name
                      AWS S3 directory name, where the input files are stored
    :param _output_dir: output directory name
                      AWS S3 directory name, where the output files are saved
    :param _data_size: size of data
                      Data size, that needs to be tested, by default it takes value of
                      -1, which means consider all the data
    :return:
            the processed data, and demand data
    """
    import os.path
    import pandas as pd
    from pandas import DataFrame
    from uszipcode import SearchEngine

    # to obtain the zip-codes for latitude, longitude values
    engine = SearchEngine()

    if not os.path.exists(_output_dir):
        os.makedirs(_output_dir)
    
    def get_zip(entry):
        return engine.by_coordinates(entry['Lat'], entry['Lon'], radius=10, returns=1)[0].zipcode

    def get_zip_codes(df):
        zip_codes = []
        zip_history = {}
        for i, entry in df.iterrows():
            if i % 1000 == 0:
                print(i)
            if (entry['Lat'], entry['Lon']) in zip_history:
                zip_codes.append(zip_history[entry['Lat'], entry['Lon']])
            else:
                zip_code = get_zip(entry)
                zip_history[entry['Lat'], entry['Lon']] = zip_code
                zip_codes.append(zip_code)
        return zip_codes
    
    # load all the data
    months = ["apr", "may", "jun", "jul", "aug", "sep"]
    file_format = "uber-raw-data-{}14.csv"
    for month in months:
        file_name = _input_dir + "/" + file_format.format(month)
        _data = pd.read_csv(file_name)
        print(len(_data))
        # obtaining the zip-codes
        print("obtaining zip for", month)
        _data['zip'] = get_zip_codes(_data)
        # process date and time
        _data['Date/Time'] = pd.to_datetime(_data['Date/Time'], format='%m/%d/%Y %H:%M:%S')
        _data['weekday'] = _data['Date/Time'].dt.dayofweek
        _data['hour'] = _data['Date/Time'].dt.hour

        # obtaining the zip-codes
        _data['zip'] = _data.apply(
            lambda row: engine.by_coordinates(row['Lat'], row['Lon'], radius=10)[0].zipcode, axis=1
        )
        output_file_name = _input_dir + "/" + file_format.format(month)
        output_file_name = output_file_name.replace("raw", "processed")
        _data.to_csv(output_file_name, index=False)

In [ ]:
from datetime import datetime
start = datetime.now()
initial_process("data", "output")
end = datetime.now()
print(end)
print(start)
print(f"initial processing time {(end - start).total_seconds()}")

564516
obtaining zip for apr
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
